In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer ## For stemming
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('punkt')  ## library for sentence tokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tiken\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
pwd

'C:\\Users\\tiken\\NLP Topic Modelling'

In [3]:
df=pd.read_csv('https://github.com/asbhupendra/dataset/blob/main/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [4]:
df.head(4)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237


In [9]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [10]:
df['Company'].value_counts()

CITIBANK, N.A.                           3226
CAPITAL ONE FINANCIAL CORPORATION        2711
BANK OF AMERICA, NATIONAL ASSOCIATION    2580
JPMORGAN CHASE & CO.                     2409
WELLS FARGO & COMPANY                    2001
                                         ... 
Redfin Corporation                          1
Finco Holding Company LLC                   1
Turner Acceptance Corp.                     1
Law Office of R. Gary Shapley               1
Action Collection Service                   1
Name: Company, Length: 2197, dtype: int64

In [11]:
complaints_df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})

In [12]:
pd.set_option('display.max_colwidth',None)
complaints_df

,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [13]:
X_train, X_hold = train_test_split(complaints_df, test_size=0.6, random_state=111)

In [14]:
X_train['Product'].value_counts()

Debt collection                8720
Credit card or prepaid card    5297
Mortgage                       3809
Checking or savings account    2822
Student loan                   1236
Vehicle loan or lease          1097
Name: Product, dtype: int64

In [15]:
stemmer = PorterStemmer()

In [23]:
def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2) ] 
#    stems = [stemmer.stem(item) for item in tokens] #Commenting this line else will take longer to execute
   return tokens

In [24]:
text = 'entire time 10 years until XX/XX/2020. XXXX makes my blood boil. I have called and was lied to told to provide my checking account information over the phone in order to turn my cell phone back on. i called at XXXX them at XXXX {$300.00} was added to my bill. \n\nScam scam scam I was told I can not call the office of the President just to write to XXXX XXXX XXXX XXXX XXXX XXXX XXXX, NM XXXX. I did three thousand times. the last letter I mailed on XX/XX/2020. Two collection agencies later. \n\nI chose to leave XXXX XXXX every time I called the XXXX supervisor would threaten me on a recorded line. I need peace of mind and a good Heart to beat inside of me. Im on a XXXX XXXX due to the stress at XXXX XXXX taking all my money 4 10 years.'
nltk.word_tokenize(text)

['entire',
 'time',
 '10',
 'years',
 'until',
 'XX/XX/2020',
 '.',
 'XXXX',
 'makes',
 'my',
 'blood',
 'boil',
 '.',
 'I',
 'have',
 'called',
 'and',
 'was',
 'lied',
 'to',
 'told',
 'to',
 'provide',
 'my',
 'checking',
 'account',
 'information',
 'over',
 'the',
 'phone',
 'in',
 'order',
 'to',
 'turn',
 'my',
 'cell',
 'phone',
 'back',
 'on',
 '.',
 'i',
 'called',
 'at',
 'XXXX',
 'them',
 'at',
 'XXXX',
 '{',
 '$',
 '300.00',
 '}',
 'was',
 'added',
 'to',
 'my',
 'bill',
 '.',
 'Scam',
 'scam',
 'scam',
 'I',
 'was',
 'told',
 'I',
 'can',
 'not',
 'call',
 'the',
 'office',
 'of',
 'the',
 'President',
 'just',
 'to',
 'write',
 'to',
 'XXXX',
 'XXXX',
 'XXXX',
 'XXXX',
 'XXXX',
 'XXXX',
 'XXXX',
 ',',
 'NM',
 'XXXX',
 '.',
 'I',
 'did',
 'three',
 'thousand',
 'times',
 '.',
 'the',
 'last',
 'letter',
 'I',
 'mailed',
 'on',
 'XX/XX/2020',
 '.',
 'Two',
 'collection',
 'agencies',
 'later',
 '.',
 'I',
 'chose',
 'to',
 'leave',
 'XXXX',
 'XXXX',
 'every',
 'time',
 'I'

In [25]:
tokenize('entire time 10 years until XX/XX/2020. XXXX makes my blood boil. I have called and was lied to told to provide my checking account information over the phone in order to turn my cell phone back on. i called at XXXX them at XXXX {$300.00} was added to my bill. \n\nScam scam scam I was told I can not call the office of the President just to write to XXXX XXXX XXXX XXXX XXXX XXXX XXXX, NM XXXX. I did three thousand times. the last letter I mailed on XX/XX/2020. Two collection agencies later. \n\nI chose to leave XXXX XXXX every time I called the XXXX supervisor would threaten me on a recorded line. I need peace of mind and a good Heart to beat inside of me. Im on a XXXX XXXX due to the stress at XXXX XXXX taking all my money 4 10 years.')

['entire',
 'time',
 'years',
 'until',
 'XX/XX/2020',
 'makes',
 'blood',
 'boil',
 'have',
 'called',
 'lied',
 'told',
 'provide',
 'checking',
 'account',
 'information',
 'over',
 'phone',
 'order',
 'turn',
 'cell',
 'phone',
 'back',
 'called',
 'them',
 '300.00',
 'added',
 'bill',
 'Scam',
 'scam',
 'scam',
 'told',
 'call',
 'office',
 'President',
 'just',
 'write',
 'three',
 'thousand',
 'times',
 'last',
 'letter',
 'mailed',
 'XX/XX/2020',
 'collection',
 'agencies',
 'later',
 'chose',
 'leave',
 'every',
 'time',
 'called',
 'supervisor',
 'would',
 'threaten',
 'recorded',
 'line',
 'need',
 'peace',
 'mind',
 'good',
 'Heart',
 'beat',
 'inside',
 'stress',
 'taking',
 'money',
 'years']

In [ ]:
TfidfVectorizer()

In [26]:
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, min_df=50, max_features=10000, use_idf=False, norm=None)
tf_vectors = vectorizer_tf.fit_transform(X_train.complaints)  ##fit using train adata and transform the train data itself

In [27]:
tf_vectors.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
tf_vectors.A ## coverts to array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
len(vectorizer_tf.get_feature_names())

2965

In [29]:
vectorizer_tf.get_feature_names()

['0.00',
 '1.00',
 '10.00',
 '100.00',
 '1000.00',
 '10000.00',
 '100000.00',
 '110.00',
 '1100.00',
 '11000.00',
 '12.00',
 '120.00',
 '1200.00',
 '12000.00',
 '130.00',
 '1300.00',
 '13000.00',
 '140.00',
 '1400.00',
 '14000.00',
 '15.00',
 '150.00',
 '1500.00',
 '15000.00',
 '160.00',
 '1600.00',
 '16000.00',
 '1681c-2',
 '1692',
 '1692g',
 '170.00',
 '1700.00',
 '180.00',
 '1800.00',
 '18000.00',
 '190.00',
 '1900.00',
 '2.00',
 '20.00',
 '200.00',
 '2000.00',
 '20000.00',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2019.',
 '2020',
 '210.00',
 '2100.00',
 '220.00',
 '2200.00',
 '230.00',
 '2300.00',
 '240.00',
 '2400.00',
 '25.00',
 '250.00',
 '2500.00',
 '25000.00',
 '260.00',
 '2600.00',
 '27.00',
 '270.00',
 '2700.00',
 '28.00',
 '280.00',
 '2800.00',
 '29.00',
 '290.00',
 '2900.00',
 '3.00',
 '30.00',
 '300.00',
 '3000.00',
 '30000.00',
 '310.00',
 '320.00',
 '3200.00',
 '3300.00',
 '340.00',
 '3400.00',
 '35.00',
 '350.00',
 '3500.00',
 '36.00',
 '360.00',
 '3600.00',
 '37

In [39]:
## its an array that is created using TFIDF basically this array is representation of Ter, frequency for each words
### in each of the document
tf_vectors.shape

(22981, 2965)

In [40]:
## Now in this array representation we are going to apply algorithm LDA

In [54]:
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=3, learning_method='online', learning_offset=50, n_jobs=-1, random_state=111)

W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

In [55]:
X_train.shape ## Train Data Set

(22981, 3)

In [56]:
tf_vectors.A.shape ## Train data into tfidf

(22981, 2965)

In [57]:
W1.argmax(axis=1)

array([1, 3, 5, ..., 1, 1, 2], dtype=int64)

In [58]:
W1.shape

(22981, 6)

In [59]:
H1[0]

array([  0.25837621,   0.17480947,   0.20574774, ..., 117.53933241,
         0.55601507,  67.45650106])

In [63]:
num_words=15

vocab = np.array(vectorizer_tf.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [ ]:
topics

In [64]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [65]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.00,0.38,0.27,0.04,0.00,0.30,1
Doc1,0.26,0.00,0.00,0.73,0.00,0.00,3
Doc2,0.01,0.17,0.01,0.16,0.01,0.65,5
Doc3,0.10,0.00,0.00,0.00,0.37,0.52,5
Doc4,0.00,0.00,0.00,0.00,0.78,0.21,4
...,...,...,...,...,...,...,...
Doc22976,0.00,0.56,0.00,0.29,0.00,0.15,1
Doc22977,0.00,0.33,0.11,0.34,0.22,0.00,3
Doc22978,0.00,0.38,0.00,0.26,0.00,0.35,1
Doc22979,0.02,0.92,0.02,0.02,0.02,0.02,1


In [66]:
X_train.head()

,complaints,Product,Company
36524,"On XX/XX/2020. Checks for the full amounts owed were sent to each of these accounts from XXXX to Capital One : Capital One ending in # XXXX {$3700.00} Capital One ending in # XXXX {$680.00} Capital One ending in # XXXX {$2600.00} Capital One ending in # XXXX {$2100.00} Capital One ending in # XXXX {$3400.00} Total payment on these accounts : {$12000.00}. \nPrior to these payoff checks sent, I made my monthly payments due XX/XX/XXXX to avoid any late fees while these checks were received and processed by Capital One. \nI later received statements showing I owed interest on each one after payment in full was received. After sending them a letter priority mail on XX/XX/2020, asking why I was paying interest on top of interest, I received a letter on XX/XX/2020 letting me know the accounts were closed per my request, which I had requested they close since I was paying in full BUT no answer to my question. I attempted to call several times prior to mailing this letter but had not success in getting anyone on the line after waiting over an hour. I went ahead and paid the interest on these accounts which added to {$280.00}, out of fear I may be hit with a late fee. I attempted to call again XX/XX/2020. After waiting over an hour for a rep on phone, I was told I had to pay the interest on interest and also a residual fee after for each one. ( how do you pay interest on interest? ) Residual fee? On accounts paid in full? They said, that was their policy. \nShortly after making full payments on these accounts, On XX/XX/XXXX, my husband lost his job due to COVID and is currently unemployed. I have struggled to pay these additional fees that I find to be an illegal act by this credit card company. I was told if I did not pay the additional fees, a late fee will be added on top. I should not have to pay interest or residual fees on a payoff amount that had not outstanding balance. I also requested a copy of their agreement stating they are allowed to do this. I never received a response or any response to any of my questions. Again, we are in no financial position at this time due to loss of unemployment to be slammed with extra fees. It is unfortunate that the timing on these payments made in full and my husband losing his job came at a worse time.",Credit card or prepaid card,CAPITAL ONE FINANCIAL CORPORATION
46281,"On or about XX/XX/XXXX I opened a Citibank Basic Checking Account after seeing an offer described by Citibank that if I opened a Basic Checking Account, deposited at least {$5000.00} within 30 days of accounting opening, and held at least that amount in the account for 60 days that within 90 days after fulfilling the conditions of the agreement I would receive a {$200.00} checking account bonus. I soon deposited approximately {$5000.00} on XX/XX/XXXX into the account. On XX/XX/XXXX I received another email update from Citibank about the offer and thanking me for enrolling in the Citibank {$200.00} checking offer and repeating the terms of the agreement. \n\nOn or about XX/XX/XXXX I utilized the Citibank Mobile Application 's chat function to inquire as to when my bonus might be depositing, as it had been well over 60 days since when I opened the account and deposited the qualifying funds on XX/XX/XXXX. 60 days since my deposit would have been around early XX/XX/XXXX. The Mobile Application representative stated they could not resolve the issue and to call their One Stop Sales Unit, which I did on XX/XX/XXXX. The representative at the One Stop Sales Unit was confused and did not know how to search for the offer clearly, saying too that often the offers are for holding money in the account for 90 days or more, and provided essentially no clarity or guidance. \n\nI withdrew {$3000.00} of the {$5000.00} I had held in the account since XX/XX/XXXX on XX/XX/XXXX, my first withdrawl from the account. I am concerned as Citibank has been very unclear on when my deposit bonus will be depositing, as tha

In [67]:
WHold = lda.transform(vectorizer_tf.transform(X_hold.complaints[:5]))

In [68]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold.complaints[:5]))]
df_doc_topic = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [69]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.03,0.16,0.00,0.81,0.00,0.00,3
Doc1,0.00,0.00,0.43,0.00,0.29,0.27,2
Doc2,0.01,0.70,0.01,0.14,0.01,0.15,1
Doc3,0.00,0.00,0.80,0.00,0.00,0.19,2
Doc4,0.00,0.76,0.14,0.00,0.00,0.10,1


In [70]:
X_hold.head()

,complaints,Product,Company
30060,"I have a business checking account at BB & T. On XX/XX/2019, I attempted to deposit a check into my account and I received a message stating that I was over my monthly mobile deposit limit. I was confused because it was the first of the month and I had not deposited any checks since the previous month. I called BB & T and they said that I couldnt deposit checks into business accounts via the mobile app even though I had done that before. \n\nI was instructed to open a personal account, into which I could deposit checks via the mobile app. I was told that if I opened the account online I would have immediate access, that I could link my personal and business accounts, and immediately be able to transfer money between them. \n\nOn XX/XX/XXXX, I opened my personal account online. Though I successfully opened online, I did not have online access as I had been promised. Because I was traveling in an area where there were no BB & T branches, I could not go into a branch until XX/XX/2019. In the intervening time, my business account became overdrawn. I had the money in my personal account to bring it back into the black, but BB & T could not make the transfer until I went into a branch. During this time, I incurred an astonishing {$320.00} in overdraft fees in my business account because I had no way of transferring money between the two accounts. \n\nWhen I went into the branch, I met with XXXX XXXX in XXXX XXXX County, Florida. She investigated the accounts and told me she would link them and that I would have online access in XXXX hours. The online access still never material, I was not able to transfer money and I received an additional {$36.00} overdraft fee. This brought the total to {$360.00} in overdraft fees. \n\nI had only opened the second account on BB & Ts advice ; I could have had my customer send a bank wire directly into business account in the first week of XXXX to keep the account positive. My TOTAL deposits between the two accounts was always positive. \n\nI requested a refund and was told they would not refund any of the fees.",Checking or savings account,BB&T CORPORATION
53473,"To who it may concern, My concern is regarding Shellpoint Mortgage Servicing Company. This company has received my monthly mortgage payments and have failed to account it as payments received. My bank has sent me proof the account was cleared and paid. This company is trying to foreclosed on my house and has sent me a letter indicating they sent my file to their Loss Mitigation dept. The person who they assigned it to is XXXX XXXX XXXX x XXXX. Their corporate number is XXXX. \n\nI have can send proof of my claim from above should you need it. Please assist.",Mortgage,"Shellpoint Partners, LLC"
35879,I contacted XXXX about fraudulent charges that were made on my account and the customer service representative told me that they wouldnt be able to issue anew card or remove my fraudulent charges since the account was closed due to non-payment. I was unaware of this these charging that were made.. To my knowledge I didnt owe a payment because the account wasnt being used.,Credit card or prepaid card,"EQUIFAX, INC."
20993,"I first applied for the Fedloan Serving program in XXXX in hopes of getting loan forgiveness since I've been paying on my student loans since late XXXX or early XXXX and I have been a public servant since XXXX, XXXX. I have never missed a payment to any of the various loan companies that owned my loans. The Fedloan Program turned me down in XXXX, saying my loans, which were originally Federal Stafford Loans, did not qualify me for the Program. In XXXX, XXXX, I was solicited by XXXX XXXX and told I could get into this program to reduce my loan payments. I was charged {$690.00} by XXXX XXXX to get me into the Fedloan Serving program. My loan payments went down to $ XXXX/month for the first year. In XXXX, XXXX, Fedloan Servicing informed me via email that I needed to recertify my "" income-drive